<a href="https://colab.research.google.com/github/gisandnes/Extreme-Gradient-Boosting-with-XGBoost_DataCamp/blob/master/Using_XGBoost_in_pipelines_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!mkdir -p data #Make folders for downloads

!wget --no-verbose https://raw.githubusercontent.com/gisandnes/Extreme-Gradient-Boosting-with-XGBoost_DataCamp/master/data/chronic_kidney_disease.csv -O ./data/chronic_kidney_disease.csv
#!wget --no-verbose https://raw.githubusercontent.com/gisandnes/Extreme-Gradient-Boosting-with-XGBoost_DataCamp/master/data/ames_unprocessed_data.csv -O ./data/ames_unprocessed_data.csv 
#!wget --no-verbose https://raw.githubusercontent.com/gisandnes/Extreme-Gradient-Boosting-with-XGBoost_DataCamp/master/data/ames_housing_trimmed_processed.csv -O ./data/ames_housing_trimmed_processed.csv

2019-01-15 10:15:18 URL:https://raw.githubusercontent.com/gisandnes/Extreme-Gradient-Boosting-with-XGBoost_DataCamp/master/data/chronic_kidney_disease.csv [43051/43051] -> "./data/chronic_kidney_disease.csv" [1]


In [0]:
!pip install -q sklearn_pandas

In [0]:
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer as Imputer

from sklearn.base import BaseEstimator, TransformerMixin

import xgboost as xgb

%matplotlib inline

### Prepare data set
age		-	age 
bp		-	blood pressure 
sg		-	specific gravity
al		-   	albumin
su		-	sugar
rbc		-	red blood cells
pc		-	pus cell
pcc		-	pus cell clumps
ba		-	bacteria
bgr		-	blood glucose random
bu		-	blood urea
sc		-	serum creatinine
sod		-	sodium
pot		-	potassium
hemo		-	hemoglobin
pcv		-	packed cell volume
wc		-	white blood cell count
rc		-	red blood cell count
htn		-	hypertension
dm		-	diabetes mellitus
cad		-	coronary artery disease
appet		-	appetite
pe		-	pedal edema
ane		-	anemia
class		-	class

In [0]:
kidney  = pd.read_csv("./data/chronic_kidney_disease.csv", header=None, names=['age','bp','sg','al','su','rbc','pc','pcc','ba','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc','htn','dm','cad','appet','pe','ane','class'])

X = kidney.loc[:,"age":"ane"]
y = kidney.loc[:,"class"]

#Rearrange columns; numeric first, then character
X = X[['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc', 'rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']]

#Numeric columns: replace ? by NaN
for name in ['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc']:
  X[name] = pd.to_numeric(X[name].replace('?', np.nan))
  #X[name] = pd.to_numeric(X[name], errors='coerce')

#Character columns: replace ? by NaN
for name in ['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']:
  X[name] = X[name].replace('?', np.nan)

In [66]:
print("Distribution: BGR \n{} \n".format(X['bgr'].describe()))
print("Distribution: RBC \n{} \n".format(X['rbc'].describe()))

Distribution: BGR 
count    356.000000
mean     148.036517
std       79.281714
min       22.000000
25%       99.000000
50%      121.000000
75%      163.000000
max      490.000000
Name: bgr, dtype: float64 

Distribution: RBC 
count        248
unique         2
top       normal
freq         201
Name: rbc, dtype: object 



In [67]:
X.head(5)

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,NaN,normal,notpresent,notpresent,yes,yes,no,good,no,no
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,NaN,normal,notpresent,notpresent,no,no,no,good,no,no
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,normal,normal,notpresent,notpresent,no,no,no,good,no,no


###Kidney disease case study I: Categorical Imputer & Kidney disease case study II: Feature Union

In [68]:
# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print("Number of nulls per column: \n{}\n".format(nulls_per_column))

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()
print("Categorical columns:     {}".format(categorical_columns))

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()
print("Non-categorical columns: {}".format(non_categorical_columns))


################################################################################################################################################
# This original DataCamp way should be replaced by the one below according to
# https://stackoverflow.com/questions/52055658/sklearn-pandas-in-a-pipeline-returns-typeerror-builtin-function-or-method-obj
################################################################################################################################################
# Apply numeric imputer
#   numeric_imputation_mapper = DataFrameMapper(
#                                               [([numeric_feature],Imputer(strategy="median")) for numeric_feature in non_categorical_columns],
#                                               input_df=True,
#                                               df_out=True
#                                              )
#   
# Apply categorical imputer
#   categorical_imputation_mapper = DataFrameMapper(
#                                                   [(category_feature, CategoricalImputer()) for category_feature in categorical_columns],
#                                                   input_df=True,
#                                                   df_out=True
#                                                  )
# Import FeatureUnion
#   from sklearn.pipeline import FeatureUnion
#   
# Combine the numeric and categorical transformations
#   numeric_categorical_union = FeatureUnion([
#                                             ("num_mapper", numeric_imputation_mapper),
#                                             ("cat_mapper", categorical_imputation_mapper)
#                                            ])

transformers = []
# Apply numeric imputer
transformers.extend([([numeric_feature],Imputer(strategy="median")) for numeric_feature in non_categorical_columns])
# Apply categorical imputer
transformers.extend([(category_feature, CategoricalImputer()) for category_feature in categorical_columns])

numeric_categorical_union = DataFrameMapper(transformers, input_df=True, df_out=True)

Number of nulls per column: 
age        9
bp        12
sg        47
al        46
su        49
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wc       106
rc       131
rbc      152
pc        65
pcc        4
ba         4
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64

Categorical columns:     ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']
Non-categorical columns: ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']


###Kidney disease case study III: Full pipeline

In [69]:
# Custom transformer to convert Pandas DataFrame into Dict (needed for DictVectorizer)
class Dictifier(BaseEstimator, TransformerMixin):      
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.to_dict('records')

# Create full pipeline
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))
                    ])

# Perform cross-validation
#cross_val_scores = cross_val_score(pipeline, kidney_data, y, scoring="roc_auc", cv=3)
cross_val_scores = cross_val_score(pipeline, X, y, scoring="roc_auc", cv=3)

# Print avg. AUC
print("3-fold AUC: ", np.mean(cross_val_scores))

3-fold AUC:  0.998637406769937


###Bringing it all together

In [82]:
#Default for XGBClassifier: 
#  learning_rate = 0.1
#  max_depth=3
#  n_estimators=100

# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1, 0.05), # 19 values
    'clf__max_depth': np.arange(3, 10, 1),          #7 values
    'clf__n_estimators': np.arange(50, 200, 50)     #3 values
}

# Perform RandomizedSearchCV
#randomized_roc_auc = RandomizedSearchCV(pipeline, param_distributions=gbm_param_grid, n_iter=2,scoring='roc_auc', cv=2, verbose=1) #As in DataCamp
randomized_roc_auc = RandomizedSearchCV(pipeline, param_distributions=gbm_param_grid, n_iter=30,scoring='roc_auc', cv=3, verbose=1, n_jobs=-1)

# Fit the estimator
randomized_roc_auc.fit(X,y)

# Compute metrics
print(randomized_roc_auc.best_score_, "\n")
print(randomized_roc_auc.best_estimator_, "\n")
print(randomized_roc_auc.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.1s


0.9987992685025816 

Pipeline(memory=None,
     steps=[('featureunion', DataFrameMapper(default=False, df_out=True,
        features=[(['age'], SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0)), (['bp'], SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbos...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]) 

{'clf__n_estimators': 100, 'clf__max_depth': 6, 'clf__learning_rate': 0.05}


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   17.8s finished


In [83]:
gbm_param_grid

{'clf__learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
        0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
 'clf__max_depth': array([3, 4, 5, 6, 7, 8, 9]),
 'clf__n_estimators': array([ 50, 100, 150])}

###Try GridSearchCV

In [84]:
#Default for XGBClassifier: 
#  learning_rate = 0.1
#  max_depth=3
#  n_estimators=100

# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1, 0.05), # 19 values
    'clf__max_depth': np.arange(3, 10, 1),          #7 values
    'clf__n_estimators': np.arange(50, 200, 50)     #3 values
}

# Perform GridSearchCV
gridsearch_roc_auc = GridSearchCV(pipeline, param_grid=gbm_param_grid,scoring='roc_auc', cv=3, verbose=1, n_jobs=-1)

# Fit the estimator
gridsearch_roc_auc.fit(X,y)

# Compute metrics
print(gridsearch_roc_auc.best_score_, "\n")
print(gridsearch_roc_auc.best_estimator_, "\n")
print(gridsearch_roc_auc.best_params_)

Fitting 3 folds for each of 399 candidates, totalling 1197 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  2.6min


0.9989595094664372 

Pipeline(memory=None,
     steps=[('featureunion', DataFrameMapper(default=False, df_out=True,
        features=[(['age'], SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0)), (['bp'], SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbos...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]) 

{'clf__learning_rate': 0.1, 'clf__max_depth': 3, 'clf__n_estimators': 50}


[Parallel(n_jobs=-1)]: Done 1197 out of 1197 | elapsed:  4.0min finished
